In [2]:
def get_sina_compasses(sid, days):
    dates = []
    closes = []
    chgrts = []
    compasses = []
    minouts = []
    sum_minout = 0
    sum_minouts = []
    try:
        headers = {
            'Referer': 'http://vip.stock.finance.sina.com.cn/moneyflow/',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36',
            'Content-type': 'application/x-www-form-urlencoded',
        }

        ssid = ''
        if sid.startswith('6'):
            ssid = 'sh'+sid
        else:
            ssid = 'sz'+sid
        params = (
            ('daima', ssid),
        )

        params = (
            ('page', '1'),
            ('num', str(days)),
            ('sort', 'opendate'),
            ('asc', '0'),
            ('daima', ssid),
        )

        response = requests.get('http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/MoneyFlow.ssl_qsfx_zjlrqs', headers=headers, params=params)
        print(response.status_code)
        resp_text = response.text
        matches = re.findall(r'[{](.*?)[}]', resp_text)
        matches.reverse()
        for data in matches:
            # DATA EXAMPLE:
            #             ['opendate:"2019-02-28"', 'trade:"4.4700"', 'changeratio:"0.0205479"', 'turnover:"277.464"',
            #              'netamount:"7978120.1700"', 'ratioamount:"0.0961108"',
            #              'r0_net:"0.0000"', 'r0_ratio:"0.00000000"', 'r0x_ratio:"0"', 'cnt_r0x_ratio:"3"',
            #              'cate_ra:"-0.00925987"', 'cate_na:"-2757227152.6700"']
            items = data.split(',')
            for item in items:
                kv = item.split(':')
                k = kv[0]
                v = kv[1]
                if k == 'opendate':
                    dates.append(v[1:-1])
                if k == 'trade':
                    closes.append(round(float(v[1:-1]), 2))
                if k == 'changeratio':
                    chgrts.append(round(float(v[1:-1]) * 100, 2))
                if k == 'r0_net':
                    mio = round(float(v[1:-1]) / 10000, 1)
                    minouts.append(mio)
                    sum_minout += mio
                    sum_minouts.append(sum_minout)
                if k == 'r0x_ratio':
                    compasses.append(round(float(v[1:-1]), 2))

    except Exception as e:
        print(sid, ' pass with error')
    return dates,closes,chgrts,compasses,minouts,sum_minouts

In [3]:
def get_mf_from_sina(sid, days):
    headers = {
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36',
        'Content-type': 'application/x-www-form-urlencoded',
        'Accept': '*/*',
        'Referer': 'http://vip.stock.finance.sina.com.cn/moneyflow/',
        'Connection': 'keep-alive',
        'If-Modified-Since': 'Tue, 26 Feb 2019 15:02:12 GMT',
    }
    ssid = ''
    if sid.startswith('6'):
        ssid = 'sh'+sid
    else:
        ssid = 'sz'+sid

    params = (
        ('page', '1'),
        ('num', str(days)),
        ('sort', 'opendate'),
        ('asc', '0'),
        ('daima', ssid),
    )
    
    dates = []
    closes = []
    chgrts = []
    slarges = []
    larges = []
    middles = []
    smalls = []
    minouts = []
    sum_mio = 0
    sum_mios = []
    mfs = []
    
    with requests.get('http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/MoneyFlow.ssl_qsfx_lscjfb', headers=headers, params=params) as response:
        resp_text = response.text
        if response.status_code != 200:
            print('HTTP Code:', response.status_code)
        else:
            matches = re.findall(r'[{](.*?)[}]', resp_text)
            matches.reverse()

            for data in matches:
                # DATA EXAMPLE:
                #             ['opendate:"2018-09-26"', 'trade:"3.5200"', 'changeratio:"0"', 'turnover:"157.832"', 'netamount:"-4620136.0200"', 'ratioamount:"-0.0701847"', 
                #              'r0:"1135515.0000"',     'r1:"26981644.3400"',     'r2:"20581001.4600"',     'r3:"17147318.0200"',
                #              'r0_net:"1135515.0000"', 'r1_net:"-3364747.9400"', 'r2_net:"-1461997.5400"', 'r3_net:"-928905.5400"']
                items = data.split(',')
                date = ''
                close = 0
                chgrt = 0
                slarge = 0
                large = 0
                middle = 0
                small = 0
                minout = 0
                for item in items:
                    kv = item.split(':')
                    k = kv[0]
                    v = kv[1][1:-1]
                    #  print(k, v)
                    if k == 'opendate':
                        date = v
                    if k == 'trade':
                        close = round(float(v), 2)
                    if k == 'changeratio':
                        chgrt = round(float(v) * 100, 2)
                    if k == 'r0_net':
                        slarge = round(float(v)/10000, 2)
                    if k == 'r1_net':
                        large = round(float(v)/10000, 2)
                    if k == 'r2_net':
                        middle = round(float(v)/10000, 2)
                    if k == 'r3_net':
                        small = round(float(v)/10000, 2)
                mfs.append(slarge + large)
                dates.append(date)
                closes.append(close)
                chgrts.append(chgrt)
                slarges.append(slarge)
                larges.append(large)
                middles.append(middle)
                smalls.append(small)
                minouts.append(minout)
                sum_mios.append(sum_mio)
            return dates, closes, chgrts, slarges, larges, middles, smalls, mfs
    return None

In [3]:
def get_mf_from_ea(sid, days):
    ssid = ''
    if sid.startswith('6'):
        ssid = sid+'1'
    else:
        ssid = sid+'2'

    headers = {
        'Referer': 'http://data.eastmoney.com/zjlx/'+sid+'.html',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36',
    }

    params = (
        ('type', 'hff'),
        ('rtntype', '2'),
        ('js', '(/{data:/[(x)/]/})'),
        ('cb', 'var aff_data='),
        ('check', 'TMLBMSPROCR'),
        ('acces_token', '1942f5da9b46b069953c873404aad4b5'),
        ('id', ssid),
        ('_', str(math.floor(time.time() * 1000))),
    )

    dates = []
    closes = []
    chgrts = []
    slarges = []
    larges = []
    middles = []
    smalls = []
    minout = []
    sum_mios = []
    try:
        response = requests.get('http://ff.eastmoney.com//EM_CapitalFlowInterface/api/js', headers=headers, params=params, timeout=1)
        resp_text = response.text
        resp_text = resp_text.replace('/', '')
        data_list = eval(resp_text[20:-2])
        sum_minout = 0
        items = data_list[0][-days:]
        for item in items:
            # DATA EXAMPLE:
            #             ['2018-09-26', '-983.9189', '-10.01%', 
            #              '-200.7831', '-2.04%', '-783.1358', '-7.97%', '-352.4439', '-3.59%', '1336.3629', '13.59%',
            #              '11.65', '-2.84%']
            day_data = item.split(',')
            date = day_data[0]
            mforce = float(day_data[1])
            slarge = float(day_data[3])
            large = float(day_data[5])
            middle = float(day_data[7])
            small = float(day_data[9])
            close = float(day_data[11])
            chgrt = float(day_data[12][:-1])
            
            dates.append(date)
            closes.append(close)
            chgrts.append(chgrt)
            slarges.append(slarge)
            larges.append(large)
            middles.append(middle)
            smalls.append(smalls)
            sum_minout += slarge + large
            sum_mios.append(sum_minout)
            
    #     print('[{:>10.2f} {:>10.2f} | {:>10.2f} {:>10.2f}] {:>6.2f} {:>5.2f}\n'.format(slarge, large, middle, small, cur_df.iloc[0]['curr_price'], cur_df.iloc[0]['curr_price'] - cur_df.iloc[0]['tday_open_price']))
    except Exception as e:
        print('Get {} failed'.format(sid))
    
    return dates, closes, chgrts, slarges, larges, middles, smalls, sum_mios
